In [1]:
from rasterstats import zonal_stats
from multiprocessing import Pool
import pandas as pd
import numpy as np
import rasterio
import geopandas as gpd
from concurrent.futures import ProcessPoolExecutor, as_completed


In [2]:
# after several trials, found that the nuts2 code used by EUROSTAT is 2013 version
nuts = gpd.read_file('/mnt/inca/tillage_index/data/EUROSTATS/nuts2/NUTS_RG_20M_2013_3035.shp')
nuts = nuts.loc[nuts['LEVL_CODE'].isin([2])]
nuts.to_file('ndti_nuts2.shp',driver='ESRI Shapefile')


stats = ['count', 'mean', 'std', 'median', 'percentile_90', 'percentile_10']
tgt = ['/mnt/inca/tillage_index/data/tif_files/ndti.min_cropland_trend.tif',
       '/mnt/inca/tillage_index/data/tif_files/min.ndti_2016.tif',
       '/mnt/inca/tillage_index/data/tif_files/min.ndti_2010.tif']

name = ['trend','2016','2010']

def calculate_zonal_stats(args):
    index, geometry, tif_path, stats = args
    zs = zonal_stats(geometry, tif_path, stats=stats)
    return index, zs

nuts = nuts.reset_index(drop=True)

In [5]:
for iii in [2, 1, 0]:
    tif_name = name[iii]
    tif_path = tgt[iii]
    
    # Prepare the dataframe for new statistics
    for stat in stats:
        nuts[f"{tif_name}_{stat}"] = np.nan
    
    # Create a list of arguments for each geometry
    args_list = [(index, geometry.to_wkt(), tif_path, stats) for index, geometry in enumerate(nuts.geometry)]

    # Use ProcessPoolExecutor to parallelize the zonal statistics calculation
    with ProcessPoolExecutor(max_workers=40) as executor:
        # Submit all the tasks and create a future object for each
        futures = [executor.submit(calculate_zonal_stats, args) for args in args_list]
        
        # As each future completes, update the dataframe
        for future in as_completed(futures):
            index, zs = future.result()
            if zs:
                for stat in stats:
                    nuts.at[index, f"{tif_name}_{stat}"] = zs[0].get(stat)


In [9]:
nuts.to_file('/mnt/inca/tillage_index/data/011_vis_ndti.shp',driver='ESRI Shapefile')

/tmp/ipykernel_1743/3009137009.py:1: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  nuts.to_file('/mnt/inca/tillage_index/data/011_vis_ndti.shp',driver='ESRI Shapefile')
